In [1]:
import pypylon.pylon as py
import cv2
import time

In [2]:
tlf = py.TlFactory.GetInstance()
devices = tlf.EnumerateDevices()
device = devices[0]
device

<pypylon.pylon.DeviceInfo; proxy of <Swig Object of type 'Pylon::CDeviceInfo *' at 0x0000021BFE885AA0> >

In [3]:
device.GetFriendlyName()

'Basler daA1440-220um (40458776)'

In [54]:
cd = tlf.CreateDevice(device)
cam = py.InstantCamera(cd)
cam.Open()

In [85]:
cam.Width.Value = 1440
cam.Height.Value = 1080
cam.AcquisitionFrameRateEnable.Value = True
cam.AcquisitionFrameRate.Value = 200
cam.ExposureTime.Value = 3500
cam.ExposureAuto.Value = "Off"
cam.DeviceLinkThroughputLimitMode.Value = "Off"

In [36]:
# One by One

tot_time = 10.
fcount = 0
cam.StartGrabbing(py.GrabStrategy_OneByOne)
stime = time.time() #has to be after setting grab strategy!
while cam.IsGrabbing() and time.time() - stime < tot_time:
    res = cam.RetrieveResult(500)
    if res.IsValid():
        fcount += 1
    res.Release()
cam.StopGrabbing()
fcount

2000

In [21]:
# Latest Image Only (for streaming)

tot_time = 10.
fcount = 0
cam.StartGrabbing(py.GrabStrategy_LatestImageOnly)
stime = time.time() #has to be after setting grab strategy!
while cam.IsGrabbing() and time.time() - stime < tot_time:
    res = cam.RetrieveResult(500)
    if res.IsValid():
        fcount += 1
    res.Release()
cam.StopGrabbing()
fcount

2000

In [32]:
# Latest Images (with Buffer)

cam.MaxNumBuffer.Value = 100

tot_time = 10.
fcount = 0
cam.StartGrabbing(py.GrabStrategy_LatestImages)
stime = time.time() #has to be after setting grab strategy!
while cam.IsGrabbing() and time.time() - stime < tot_time:
    res = cam.RetrieveResult(500)
    if res.IsValid():
        fcount += 1
    res.Release()
cam.StopGrabbing()
fcount

2000

In [83]:
cam.ResultingFrameRate.Value

200.0

In [88]:
# One by One
fourcc = cv2.VideoWriter_fourcc(*'H264')
video_writer = cv2.VideoWriter('output_OO.avi', fourcc, fps=200, frameSize=(1440, 1080), isColor=False)

tot_time = 10.
fcount = 0
cam.StartGrabbing(py.GrabStrategy_OneByOne)
stime = time.time() #has to be after setting grab strategy!
while cam.IsGrabbing() and time.time() - stime < tot_time:
    res = cam.RetrieveResult(1000)
    if res.GrabSucceeded(): #use this instead of IsValid!
        fcount += 1
        #image = cv2.cvtColor(res.Array, cv2.COLOR_GRAY2BGR)
        image = res.Array
        video_writer.write(image)
    res.Release()
cam.StopGrabbing()
video_writer.release()
fcount

1999

In [89]:
# Latest Image Only (for streaming)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_writer = cv2.VideoWriter('output_LI.avi', fourcc, fps=200, frameSize=(1440, 1080), isColor=False)

tot_time = 10.
fcount = 0
cam.StartGrabbing(py.GrabStrategy_LatestImageOnly)
stime = time.time() #has to be after setting grab strategy!
while cam.IsGrabbing() and time.time() - stime < tot_time:
    res = cam.RetrieveResult(1000)
    if res.GrabSucceeded(): #use this instead of IsValid!
        fcount += 1
        #image = cv2.cvtColor(res.Array, cv2.COLOR_GRAY2BGR)
        image = res.Array
        video_writer.write(image)
    res.Release()
cam.StopGrabbing()
video_writer.release()
fcount

1817

In [76]:
# Latest Images (with Buffer)
fourcc = cv2.VideoWriter_fourcc(*'XVID')
video_writer = cv2.VideoWriter('output_Buffer.avi', fourcc, fps=200, frameSize=(1440, 1080), isColor=False)

cam.MaxNumBuffer.Value = 100

tot_time = 10.
fcount = 0
cam.StartGrabbing(py.GrabStrategy_LatestImages)
stime = time.time() #has to be after setting grab strategy!
while cam.IsGrabbing() and time.time() - stime < tot_time:
    res = cam.RetrieveResult(500)
    if res.GrabSucceeded(): #IsValid seemed to work here
        fcount += 1
        #image = cv2.cvtColor(res.Array, cv2.COLOR_GRAY2BGR)
        image = res.Array
        video_writer.write(image)
    res.Release()
cam.StopGrabbing()
video_writer.release()
fcount

1788

In [50]:
cam.Close()